In [ ]:
%env DASHSCOPE_API_KEY=请到千问灵积官方（https://dashscope.console.aliyun.com/apiKey）申请免费Key，并替换为自己的
%env TAVILY_API_KEY=请到Tavily官方（https://app.tavily.com/）申请免费Key，并替换为自己的
%env AMAP_API_KEY=请到高德开发者中心（https://console.amap.com/dev/index），申请免费key，并替换为自己的

In [12]:
!pip install -qU dashscope

In [13]:
import langchain, langchain_core, langchain_community, dashscope

for module in (langchain, langchain_core, langchain_community, dashscope):
    if hasattr(module, '__version__'):
        print(f"{module.__name__:<20}\t{module.__version__}")

langchain           	0.2.3
langchain_core      	0.2.5
langchain_community 	0.2.1


In [14]:
!pip list | grep -i dashscope

dashscope                                1.19.2


# 绑定工具

In [15]:
import os
from langchain_community.chat_models.tongyi import ChatTongyi
from langchain_core.tools import tool

使用通义千问官方API

In [61]:
model = ChatTongyi(model='qwen-turbo')

工具定义及工具绑定

In [69]:
def search_web(query, k=5):
    import os
    from langchain_community.retrievers import TavilySearchAPIRetriever
    
    retriever = TavilySearchAPIRetriever(k=5)
    documents = retriever.invoke(query)
    # return [{'title': doc.metadata['title'], 'abstract': doc.page_content, 'href': doc.metadata['source'], 'score': doc.metadata['score']} for doc in
    #         documents]
    content = '\n\n'.join([doc.page_content for doc in documents])
    prompt = f"""请将下面这段内容（<<<content>>><<</content>>>包裹的部分）进行总结：
<<<content>>>
{content}
<<</content>>>
"""
    print('prompt:')
    print(prompt)
    
    return model.invoke(prompt).content
    

def get_current_weather(city):
    import requests
    from datetime import datetime
    import os
    
    amap_api_key = os.environ['AMAP_API_KEY']
    url = f'https://restapi.amap.com/v3/config/district?keywords={city}&key={amap_api_key}'
    resp = requests.get(url)
    # print('行政区划：')
    # print(resp.json())
    
    adcode = resp.json()['districts'][0]['adcode']
    # adcode = '110000'

    url = f'https://restapi.amap.com/v3/weather/weatherInfo?city={adcode}&key={amap_api_key}&extensions=base'
    resp = requests.get(url)
    """
    {'province': '北京',
     'city': '北京市',
     'adcode': '110000',
     'weather': '晴',
     'temperature': '26',
     'winddirection': '西北',
     'windpower': '4',
     'humidity': '20',
     'reporttime': '2024-05-26 13:38:38',
     'temperature_float': '26.0',
     'humidity_float': '20.0'}
    """
    # print('天气：')
    # print(resp.json())
    weather_json = resp.json()['lives'][0]
    return f"{weather_json['city']}{datetime.strptime(weather_json['reporttime'], '%Y-%m-%d %H:%M:%S').strftime('%m月%d日')}{weather_json['weather']}，气温{weather_json['temperature']}摄氏度，{weather_json['winddirection']}风{weather_json['windpower']}级"
    
    
fn_map = {
    'get_current_weather': get_current_weather,
    'search_web': search_web
}

llm_with_tool = model.bind_tools(
    tools=[
        {
            "name": "get_current_weather",
            "description": "根据城市名获取天气",
            "parameters": {
                "type": "object",
                "properties": {
                    "city": {
                        "type": "string",
                        "description": "城市名，例如北京"
                    }
                },
                "required": ["city"]
            }
        },
        {
            "name": "search_web",
            "description": "当有不知道或不确定的信息时，搜索互联网",
            "parameters": {
                "type": "object",
                "properties": {
                    "query": {
                        "type": "string",
                        "description": "要搜素的内容"
                    }
                },
                "required": ["query"]
            }
        },
    ]
)

# 使用工具

## 天气查询

In [63]:
from langchain_core.messages import HumanMessage
import json

ai_msg = llm_with_tool.invoke("北京今天的天气怎么样")
fn_name = ai_msg.additional_kwargs['tool_calls'][0]['function']['name']
kwargs = json.loads(ai_msg.additional_kwargs['tool_calls'][0]['function']['arguments'])

In [64]:
ai_msg

AIMessage(content='', additional_kwargs={'tool_calls': [{'function': {'name': 'get_current_weather', 'arguments': '{"city": "北京"}'}, 'id': '', 'type': 'function'}]}, response_metadata={'model_name': 'qwen-turbo', 'finish_reason': 'tool_calls', 'request_id': '323072e8-af31-9696-9366-8f0092fcef38', 'token_usage': {'input_tokens': 255, 'output_tokens': 18, 'total_tokens': 273}}, id='run-5cd79a87-9804-444f-b20c-643ddd47edcb-0', tool_calls=[{'name': 'get_current_weather', 'args': {'city': '北京'}, 'id': ''}])

从结果看，触发函数调用了

In [66]:
ai_msg.additional_kwargs

{'tool_calls': [{'function': {'name': 'get_current_weather',
    'arguments': '{"city": "北京"}'},
   'id': '',
   'type': 'function'}]}

In [70]:
fn_map[fn_name](**kwargs)

'北京市06月09日多云，气温29摄氏度，东南风≤3级'

## llm_with_tool.invoke实际发送的请求

llm_with_tool.invoke("北京今天的天气怎么样")这一句，实际在底层发送请求时，调用的是dashscope库中，dashscope/api_entities/http_request.py这个文件的_handle_request方法

发送的是一个HTTP POST方法，具体内容如下：

```json
{
    "model": "qwen-turbo",
    "parameters": {
        "top_p": 0.8,
        "result_format": "message",
        "tools": [
            {
                "type": "function",
                "function": {
                    "name": "get_current_weather",
                    "description": "根据城市名获取天气",
                    "parameters": {
                        "type": "object",
                        "properties": {
                            "city": {
                                "type": "string",
                                "description": "城市名，例如北京"
                            }
                        },
                        "required": [
                            "city"
                        ]
                    }
                }
            },
            {
                "type": "function",
                "function": {
                    "name": "search_web",
                    "description": "当有不知道或不确定的信息时，搜索互联网",
                    "parameters": {
                        "type": "object",
                        "properties": {
                            "query": {
                                "type": "string",
                                "description": "要搜素的内容"
                            }
                        },
                        "required": [
                            "query"
                        ]
                    }
                }
            }
        ]
    },
    "input": {
        "messages": [
            {
                "role": "user",
                "content": "北京今天的天气怎么样"
            }
        ]
    }
}
```

## 网络搜索

In [71]:
ai_msg = llm_with_tool.invoke("庆余年2好看吗")

In [72]:
ai_msg

AIMessage(content='关于电视剧《庆余年2》是否好看，这是一个主观评价问题，不同观众可能有不同的看法。《庆余年》第一部在2019年播出时受到了很多好评，但第二部的消息目前还没有正式定档和发布详细的剧情介绍，所以无法直接判断它的质量。\n\n如果你对剧集的续集感兴趣，可以关注官方消息或者查看网上的预告片和评论，以了解大致的剧情走向和制作水平。如果你想要了解更多关于《庆余年》的详细信息或者对某个具体方面有疑问，比如演员阵容、剧情改编等，我可以帮你搜索相关资料。', response_metadata={'model_name': 'qwen-turbo', 'finish_reason': 'stop', 'request_id': 'ef1555fb-2545-9c7f-b176-5d8cab85988d', 'token_usage': {'input_tokens': 258, 'output_tokens': 128, 'total_tokens': 386}}, id='run-aa88f959-727b-45e4-b925-d19472839c65-0')

从结果看，并没有触发函数调用

In [73]:
ai_msg = llm_with_tool.invoke("请帮我搜索并回答庆余年2好看吗")

In [74]:
ai_msg

AIMessage(content='', additional_kwargs={'tool_calls': [{'function': {'name': 'search_web', 'arguments': '{"query": "庆余年2 好看吗"}'}, 'id': '', 'type': 'function'}]}, response_metadata={'model_name': 'qwen-turbo', 'finish_reason': 'tool_calls', 'request_id': 'b92aa0b6-573c-9d3a-ab8c-8c31c64fd115', 'token_usage': {'input_tokens': 263, 'output_tokens': 25, 'total_tokens': 288}}, id='run-edd276b9-deb5-4be0-bd47-d79c7797856a-0', tool_calls=[{'name': 'search_web', 'args': {'query': '庆余年2 好看吗'}, 'id': ''}])

在query中显式指定需要“搜索”，成功触发函数调用

In [75]:
fn_name = ai_msg.additional_kwargs['tool_calls'][0]['function']['name']
kwargs = json.loads(ai_msg.additional_kwargs['tool_calls'][0]['function']['arguments'])

In [78]:
fn_name

'search_web'

In [77]:
kwargs

{'query': '庆余年2 好看吗'}

从query参数来看，LLM对输入做了分词，并将其转换成了适合搜索的关键词形式

调用函数

In [79]:
fn_map[fn_name](**kwargs)

prompt:
请将下面这段内容（<<<content>>><<</content>>>包裹的部分）进行总结：
<<<content>>>
庆余年2开播这天，我给我自己放了一首《好日子》。 因为点开第一集刚看了个开头，我就知道，属于我的2019年重新杀了回来。 故事的接续感诚意十足，一点开就是张庆被催更，回扣整体的世界观，再跟随故事回到剧中，以同样的节奏同样的视角重现第一季的 ...

观众的打分和评价都一针见血_腾讯新闻. 《庆余年2》第一波真实口碑出炉!. 观众的打分和评价都一针见血. 等了5年，终于等到《 庆余年2 》惊喜开播了。. 而这部剧的首播当晚排面有多大呢？. 全网热度第一，热搜前十八个都和这部剧有关系。. 这个架势，不愧 ...

庆余年2开播这天，我给我自己放了一首《好日子》。 因为点开第一集刚看了个开头，我就知道，属于我的2019年重新杀了回来。 故事的接续感诚意十足，一点开就是张庆被催更，回扣整体的世界观，再跟随故事回到剧中，以同样的节奏同样的视角重现第一季的 ...

等了五年，《庆余年2》让观众们看了个寂寞。 开播之前，各种宣传把这部剧说的是天花乱坠，又是原班人马全员回归，又是名场面不断。 由于第二季的体量仅有36集，因此很难不让人以为《庆余年2》的节奏很快，一定会单刀直入的开始讲第二季的情节。

庆余年2前5集观后感，优缺点浅评. 1～2集：前两集不太好看，因为看庆2之前我刚把第一季精编版刷完重温了一下剧情，感觉节奏和味道都不太对，没有庆1的节奏感，剧情很平，梗多，但是信息量很低，有些水，假死剧情逻辑上来说圆得很生硬，没什么必要 ...
<<</content>>>



'在《庆余年2》开播当天，观众的感受强烈，有人通过歌曲表达了期待和回归，认为故事有很好的接续感和世界观一致性。然而，首播时观众反馈存在一些问题，如与第一季节奏不符、剧情平淡、信息量低以及部分情节处理生硬等。尽管前期宣传热烈，实际观看体验并不完全符合期待，尤其是对于刚重温过第一季的观众来说。总体而言，首播成绩显著但具体内容存在争议。'